In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from Board import Board
from Filewriter import Filewriter

In [ ]:
fw = Filewriter()

In [ ]:
def basicCheck(hexStr):
    expect = ""
    for i in range(10):
        expect += str(i)*6
    expect += 'aaaaaabbbbbb'
    return ''.join(sorted(hexStr.lower())) == expect

def getSolution():
    hexStr = ''.join(solution.split()) 
    b.drawBoard([int(c, 12) for c in hexStr])
    plt.show()
    print(id)
    print(discoverer)
    print("Date: "+str(date))
    if not basicCheck(hexStr):
        print("WARNING: Did not pass basic check")
    elif raw_input("Press enter if good ") == "":
        fw.addNewSolution(hexStr, discoverer, date, id)

In [ ]:
b = Board() # needs to be in same cell for some reason for board to show up

id = 10
discoverer = "Corina Miner"
date = None
# date = "2016-1-26"
# date = "2018-07-26"
solution = """
        111
       21211
      7222200
     477770000
    4444b7a6666
   55548bbaaa366
   55588bbbaa333
    88899999933
    """
getSolution()